In [7]:
import pandas as pd
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [11]:

# File name for storing the vital signs
FILE_NAME = 'tracked_vital_signs.csv'
DATASET_FILE = 'human_vital_signs_dataset_2024.csv'  # Use the provided dataset file name

# Load dataset for training from the provided CSV file
def load_dataset():
    # Load the dataset from the provided file name
    df = pd.read_csv(DATASET_FILE)
    df.columns = df.columns.str.strip()  # Trim whitespace from column names
    print("Dataset columns:", df.columns.tolist())  # Print the columns to verify
    return df

# Train an ML model
def train_ml_model():
    # Load the dataset
    df = load_dataset()

    # Check if expected columns are present
    required_columns = ['Heart Rate', 'Respiratory Rate', 'Body Temperature', 'Oxygen Saturation', 'Systolic BP', 'Diastolic BP', 'Risk Category']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in dataset. Available columns: {df.columns.tolist()}")

    # Features and target
    X = df[['Heart Rate', 'Respiratory Rate', 'Body Temperature', 'Oxygen Saturation', 'Systolic BP', 'Diastolic BP']]
    y = df['Risk Category']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Test the model's accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Model accuracy: {accuracy:.2f}')

    return model

# Function to predict risk using the trained ML model
def predict_risk_with_ml(model, heart_rate, respiratory_rate, body_temperature, oxygen_saturation, systolic_bp, diastolic_bp):
    # Prepare the input data for the model
    input_data = [[heart_rate, respiratory_rate, body_temperature, oxygen_saturation, systolic_bp, diastolic_bp]]

    # Predict the risk category (0 = Low Risk, 1 = High Risk)
    prediction = model.predict(input_data)

    return "High Risk" if prediction[0] == 1 else "Low Risk"

# Function to save the vital signs to a CSV file
def save_to_csv_ml(data):
    if not os.path.isfile(FILE_NAME):
        # Create a new DataFrame and save as CSV if it doesn't exist
        df = pd.DataFrame(data, columns=[
            'Timestamp', 'Heart Rate', 'Respiratory Rate', 'Body Temperature', 
            'Oxygen Saturation', 'Systolic BP', 'Diastolic BP', 'MAP', 'Risk Category'
        ])
        df.to_csv(FILE_NAME, index=False)
    else:
        # Append the data to the existing CSV file
        df = pd.DataFrame(data, columns=[
            'Timestamp', 'Heart Rate', 'Respiratory Rate', 'Body Temperature', 
            'Oxygen Saturation', 'Systolic BP', 'Diastolic BP', 'MAP', 'Risk Category'
        ])
        df.to_csv(FILE_NAME, mode='a', header=False, index=False)

# Function to track and predict the risk using the ML model
def track_vital_signs_ml(model, duration=60):
    end_time = time.time() + duration
    data = []

    while time.time() < end_time:
        # Input vital signs from the user
        heart_rate = int(input("Enter Heart Rate (bpm): "))
        respiratory_rate = int(input("Enter Respiratory Rate (breaths/min): "))
        body_temperature = float(input("Enter Body Temperature (°C): "))
        oxygen_saturation = int(input("Enter Oxygen Saturation (%): "))
        systolic_bp = int(input("Enter Systolic Blood Pressure (mmHg): "))
        diastolic_bp = int(input("Enter Diastolic Blood Pressure (mmHg): "))

        # Derived MAP
        map_value = round((2 * diastolic_bp + systolic_bp) / 3, 2)

        # Predict risk using the ML model
        risk_category = predict_risk_with_ml(model, heart_rate, respiratory_rate, body_temperature, oxygen_saturation, systolic_bp, diastolic_bp)
        print(f"Predicted Risk Category: {risk_category}")

        # Timestamp
        timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
        data.append([timestamp, heart_rate, respiratory_rate, body_temperature, oxygen_saturation, systolic_bp, diastolic_bp, map_value, risk_category])

        # Save to CSV
        save_to_csv_ml(data)

        # Sleep for 10 seconds before the next reading
        time.sleep(10)

    print("Monitoring completed!")

# Function to display the tracked vital signs
def display_vital_signs_ml():
    if os.path.isfile(FILE_NAME):
        df = pd.read_csv(FILE_NAME)
        print("\nTracked Vital Signs:")
        print(df)
    else:
        print("No data tracked yet.")

# Main function to train the model and start tracking
def main():
    # Train the ML model
    model = train_ml_model()

    # Start tracking vital signs using the trained ML model
    track_duration = int(input("Enter tracking duration in seconds: "))
    track_vital_signs_ml(model, track_duration)
    
    # Display the tracked vital signs
    display_vital_signs_ml()

if __name__ == '__main__':
    main()


Dataset columns: ['Patient ID', 'Heart Rate', 'Respiratory Rate', 'Timestamp', 'Body Temperature', 'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Derived_HRV', 'Derived_Pulse_Pressure', 'Derived_BMI', 'Derived_MAP', 'Risk Category']


KeyError: "Column 'Systolic BP' not found in dataset. Available columns: ['Patient ID', 'Heart Rate', 'Respiratory Rate', 'Timestamp', 'Body Temperature', 'Oxygen Saturation', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Derived_HRV', 'Derived_Pulse_Pressure', 'Derived_BMI', 'Derived_MAP', 'Risk Category']"